# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [42]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [43]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,qaqortoq,60.7167,-46.0333,277.39,89,32,1.54,GL,1721084344
1,1,nacala,-14.5428,40.6728,293.43,82,8,2.58,MZ,1721084346
2,2,port-aux-francais,-49.3500,70.2167,275.01,78,89,13.99,TF,1721084347
3,3,serdar,38.9764,56.2757,299.08,48,0,3.34,TM,1721084349
4,4,bethel,41.3712,-73.4140,304.26,76,0,1.54,US,1721084350


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [44]:
%%capture --no-display

import geoviews.tile_sources as gvts
import geoviews as gv

# Configure the map plot

# Define the dimensions for the data
humidity = gv.Dimension('Humidity')

# Create a GeoViews Dataset from the city_data_df DataFrame
city_points = gv.Points(city_data_df, ['Lng', 'Lat'], ['City', 'Country', 'Humidity'])

# Create the map with points where the size of the point is based on the humidity
map_plot = city_points.opts(
    size=gv.dim('Humidity')*0.1,
    color=gv.dim('City'),  # Map the 'City' dimension to the colors of the points
#    cmap='plasma', 
    tools=['hover'], 
  #  color=gv.dim('blue'),
    width=1000,
    height=600,
    cmap='tab20'
  
) * gv.tile_sources.OSM

# Display the map
map_plot

:Overlay
   .Points.I :Points   [Lng,Lat]   (City,Country,Humidity)
   .WMTS.I   :WMTS   [Longitude,Latitude]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [45]:
# Narrow down cities that fit criteria and drop any results with null values
# Filter the city_data_df DataFrame
ideal_cities_df = city_data_df[
    ((city_data_df["Lat"] > 50) | (city_data_df["Lat"] < -35)) & 
    ((city_data_df["Lng"] < 20) | (city_data_df["Lng"] > 140)) &
    (((city_data_df["Max Temp"]-273.15)*9/5+32) < 85)
    ]

# Drop any rows with null values
ideal_cities_df.dropna(inplace=True)

# Display sample data
ideal_cities_df

C:\Users\johns\AppData\Local\Temp\ipykernel_30688\1367459158.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_cities_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,qaqortoq,60.7167,-46.0333,277.39,89,32,1.54,GL,1721084344
7,7,puerto natales,-51.7236,-72.4875,276.17,91,79,2.18,CL,1721084355
11,11,blackmans bay,-43.0167,147.3167,283.56,95,76,4.96,AU,1721084361
12,12,olonkinbyen,70.9221,-8.7187,279.15,90,33,2.99,SJ,1721084362
13,13,labrador city,52.9463,-66.9114,292.10,88,100,2.06,CA,1721084364
...,...,...,...,...,...,...,...,...,...,...
547,547,ketchikan,55.3422,-131.6461,288.41,94,100,6.69,US,1721085210
553,553,lachendorf,52.6167,10.2500,293.83,64,75,2.57,DE,1721085221
555,555,ancud,-41.8697,-73.8203,281.43,97,100,4.78,CL,1721085223
567,567,egilsstadir,65.2653,-14.3948,280.10,93,100,6.17,IS,1721085243


### Step 3: Create a new DataFrame called `hotel_df`.

In [46]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,qaqortoq,GL,60.7167,-46.0333,89,
7,puerto natales,CL,-51.7236,-72.4875,91,
11,blackmans bay,AU,-43.0167,147.3167,95,
12,olonkinbyen,SJ,70.9221,-8.7187,90,
13,labrador city,CA,52.9463,-66.9114,88,
...,...,...,...,...,...,...
547,ketchikan,US,55.3422,-131.6461,94,
553,lachendorf,DE,52.6167,10.2500,64,
555,ancud,CL,-41.8697,-73.8203,97,
567,egilsstadir,IS,65.2653,-14.3948,93,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [47]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,  # Limit the search to return only the nearest hotel
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    #params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    print(params)

    # Convert the API response to JSON format
    name_address = name_address.json() 

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{'categories': 'accommodation.hotel', 'limit': 1, 'apiKey': '21cf05dbe0ac4d03baf225add1dd5595', 'filter': 'circle:-46.0333,60.7167,10000'}
qaqortoq - nearest hotel: Hotel Qaqortoq
{'categories': 'accommodation.hotel', 'limit': 1, 'apiKey': '21cf05dbe0ac4d03baf225add1dd5595', 'filter': 'circle:-72.4875,-51.7236,10000'}
puerto natales - nearest hotel: Hotel Hallef
{'categories': 'accommodation.hotel', 'limit': 1, 'apiKey': '21cf05dbe0ac4d03baf225add1dd5595', 'filter': 'circle:147.3167,-43.0167,10000'}
blackmans bay - nearest hotel: Kingston Hotel
{'categories': 'accommodation.hotel', 'limit': 1, 'apiKey': '21cf05dbe0ac4d03baf225add1dd5595', 'filter': 'circle:-8.7187,70.9221,10000'}
olonkinbyen - nearest hotel: No hotel found
{'categories': 'accommodation.hotel', 'limit': 1, 'apiKey': '21cf05dbe0ac4d03baf225add1dd5595', 'filter': 'circle:-66.9114,52.9463,10000'}
labrador city - nearest hotel: Northern Inn & Suites
{'categories': 'accommodation.hotel', 'limit': 1, 'ap

,City,Country,Lat,Lng,Humidity,Hotel Name
0,qaqortoq,GL,60.7167,-46.0333,89,Hotel Qaqortoq
7,puerto natales,CL,-51.7236,-72.4875,91,Hotel Hallef
11,blackmans bay,AU,-43.0167,147.3167,95,Kingston Hotel
12,olonkinbyen,SJ,70.9221,-8.7187,90,No hotel found
13,labrador city,CA,52.9463,-66.9114,88,Northern Inn & Suites
...,...,...,...,...,...,...
547,ketchikan,US,55.3422,-131.6461,94,Cape Fox Lodge
553,lachendorf,DE,52.6167,10.2500,64,Hotel Am Kloster
555,ancud,CL,-41.8697,-73.8203,97,Mirador Quempillen
567,egilsstadir,IS,65.2653,-14.3948,93,Gistihúsið


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [48]:
%%capture --no-display

# Create a GeoViews Dataset from the city_data_df DataFrame
hotel_points = gv.Points(hotel_df, ['Lng', 'Lat'], ['City', 'Country', 'Humidity', 'Hotel Name'])

# Create the map with points where the size of the point is based on the humidity
map_plot = hotel_points.opts(
    size=gv.dim('Humidity')*0.1,
    color=gv.dim('City'),  # Map the 'City' dimension to the colors of the points
#    cmap='plasma', 
    tools=['hover'], 
  #  color=gv.dim('blue'),
    width=1000,
    height=600,
    cmap='tab20'
  
) * gv.tile_sources.OSM

# Display the map
map_plot

:Overlay
   .Points.I :Points   [Lng,Lat]   (City,Country,Humidity,Hotel Name)
   .WMTS.I   :WMTS   [Longitude,Latitude]